# What are we going to do?

We will build a data processing pipeline that takes our data sources performs aggregations by precise boundaries. Then run a weighted algorithm to calculate a "cultural" score for each city normalized by its total population. So the pipeline is something like this:

![](images/sketch.png)

## What do we mean by culture/cultural score?

This is a completely subjective and our algorithm is therefore only meant as an excuse to have fun learning PySpark and Spatial Analysis. We collect data from OpenStreetMap that seems culturally relevant. These include the following tags:

    tourism"="artwork"
    tourism"="gallery"
    amenity"="theatre"
    amenity"="arts_centre"
    tourism"="museum"

We will also assign different weights to each tag. From 1 to 5. The tags above are listed from highest (artwork) to lowest (museum). This should make our analysis interesting. We will calculate a score by performing a weighted count for each of the sites found per city. Finally we will normalize our count by dividing it by the total population of the city.

In [4]:
%load './code/helpers/imports.py'

In [6]:
%load './code/helpers/osm2geojson.py'

In [8]:
%load './code/helpers/query_overpass_api.py'

In [10]:
OVERPASS_API         = overpy.Overpass()
BASE_DIR             = os.path.join(os.path.abspath('.'), 'work-flow')
URBAN_BOUNDARIES_FILE = '06_Europe_Cities_Boundaries_with_Labels_Population.geo.json'

# Paths to base datasets that we are using:
URBAN_BOUNDARIES_PATH = os.path.join(BASE_DIR,URBAN_BOUNDARIES_FILE)
POIS_PATH            = os.path.join(BASE_DIR, "pois.json")